In [48]:
import numpy as np

# Model A
![](ModelA.png)

In [49]:
L,M1,M2,M3,M4,I1,I2,I3,C,R,T=tuple(range(11))

In [50]:
#folded probabilities
A = np.zeros((11,11))
B = np.zeros((11,3))

In [51]:
S_folded = np.zeros(11)
S_folded[M1] = 0.6
S_folded[M2] = 0.1+0.1*0.8
S_folded[M3] = 0.1+0.1*0.2*0.5
S_folded[M4] = 0.1+0.1*0.2*0.5*0.8
S_folded[C] = 0.1*0.2*0.5*0.2*0.2
S_folded[R] = 0.1*0.2*0.5*0.2*0.7
S_folded[T] = 0.1*0.2*0.5*0.2*0.1

In [52]:
init = 0.5 * S_folded
init[L] = 0.5

In [53]:
A[L,:] = 0.4 * S_folded
A[L,L] = 0.6
A[M1,M2] = 0.97
A[M1,M3] = 0.01*0.5
A[M1,M4] = 0.01*0.5*0.8
A[M1,I1] = 0.01
M1_E = 0.01*0.5*0.2 + 0.01
A[M1,C] = 0.2 * M1_E
A[M1,R] = 0.7 * M1_E
A[M1,T] = 0.1 * M1_E
A[M2,M3] = 0.5
A[M2,M4] = 0.4*0.8
A[M2,I2] = 0.05
M2_E = 0.4*0.2 + 0.05
A[M2,C] = 0.2 * M2_E
A[M2,R] = 0.7 * M2_E
A[M2,T] = 0.1 * M2_E
A[M3,M4] = 0.6
A[M3,I3] = 0.3
M3_E = 0.05*2
A[M3,C] = 0.2 * M3_E
A[M3,R] = 0.7 * M3_E
A[M3,T] = 0.1 * M3_E
A[M4,C] = 0.2
A[M4,R] = 0.7
A[M4,T] = 0.1
A[I1,I1] = 0.5
A[I1,M2] = 0.5
A[I2,I2] = 0.5
A[I2,M3] = 0.5
A[I3,I3] = 0.5
A[I3,M4] = 0.5
A[C,:] = 0.1 * S_folded
A[C,C] = 0.9 + 0.1*0.1*0.2*0.5*0.2*0.2
A[R,R] = 0.6
A[R,T] = 0.4
A[T,T] = 1

In [54]:
B[:,:2] = 0.5
B[M2,:2] = (0.1, 0.9)
B[M3,:2] = (0.7, 0.3)
B[M4,:2] = (0.9, 0.1)
B[T,:] = (0,0,1)

In [55]:
np.testing.assert_almost_equal(np.sum(init), 1)
np.testing.assert_almost_equal(np.sum(A, -1), 1)
np.testing.assert_almost_equal(np.sum(B, -1), 1)

In [56]:
A,B,init

(array([[6.0000e-01, 2.4000e-01, 7.2000e-02, 4.4000e-02, 4.3200e-02,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 1.6000e-04, 5.6000e-04,
         8.0000e-05],
        [0.0000e+00, 0.0000e+00, 9.7000e-01, 5.0000e-03, 4.0000e-03,
         1.0000e-02, 0.0000e+00, 0.0000e+00, 2.2000e-03, 7.7000e-03,
         1.1000e-03],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 5.0000e-01, 3.2000e-01,
         0.0000e+00, 5.0000e-02, 0.0000e+00, 2.6000e-02, 9.1000e-02,
         1.3000e-02],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 6.0000e-01,
         0.0000e+00, 0.0000e+00, 3.0000e-01, 2.0000e-02, 7.0000e-02,
         1.0000e-02],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 2.0000e-01, 7.0000e-01,
         1.0000e-01],
        [0.0000e+00, 0.0000e+00, 5.0000e-01, 0.0000e+00, 0.0000e+00,
         5.0000e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e

In [57]:
def forward(seq, init, A, B, scaled=False):
    num_states = init.shape[0]
    length = seq.shape[0]
    alpha = np.zeros((length, num_states))
    alpha[0] = init * B.dot(seq[0])
    if scaled:
        lik = np.sum(alpha[0])
        alpha[0] /= lik
    for i in range(1, length):
        alpha[i] = np.transpose(A).dot(alpha[i-1]) * B.dot(seq[i])
        if scaled:
            S = np.sum(alpha[i])
            alpha[i] /= S
            lik *= S
    if scaled:
        return alpha, lik
    else:
        return alpha

def backward(seq, init, A, B):
    num_states = init.shape[0]
    length = seq.shape[0]
    beta = np.zeros((length, num_states))
    beta[-1] = 1
    for i in range(1, length):
        b = B.dot(seq[length-i]) * beta[length-i]
        beta[length-1-i] = A.dot(b)
    return beta

def viterbi_variables(seq, init, A, B):
    num_states = init.shape[0]
    length = seq.shape[0]
    gamma = np.zeros((length, num_states))
    gamma[0] = init * B.dot(seq[0])
    for i in range(1, length):
        d = A * gamma[i-1][:,np.newaxis]
        gamma[i] = np.max(d, axis=0) * B.dot(seq[i])
    return gamma

def viterbi_path(gamma, A):
    length = gamma.shape[0]
    path = np.zeros(length, dtype=int)
    path[-1] = np.argmax(gamma[-1])
    for i in range(1, length):
        path[-1-i] = np.argmax(A[:,path[-i]] * gamma[-i-1])
    return path

def likelihood_forward(alpha):
    return np.sum(alpha[-1])

def likelihood_backward(beta, seq, init, B):
    return np.sum(init * B.dot(seq[0]) * beta[0])

def posterior(alpha, beta, lik):
    return alpha * beta / lik

In [58]:
seq = [0,1,0,2]
alphabet_size = B.shape[-1]
seq = np.eye(alphabet_size)[seq]

In [59]:
alpha = forward(seq, init, A, B)
beta = backward(seq, init, A, B)
lik = likelihood_forward(alpha)
lik2 = likelihood_backward(beta, seq, init, B)
np.testing.assert_almost_equal(lik, lik2)
posterior_probs = posterior(alpha, beta, lik)
alpha_scaled, lik3 = forward(seq, init, A, B, scaled=True)
np.testing.assert_almost_equal(lik, lik3)
gamma = viterbi_variables(seq, init, A, B)
optimal_path = viterbi_path(gamma, A)

In [60]:
alpha

array([[2.50000000e-01, 1.50000000e-01, 9.00000000e-03, 3.85000000e-02,
        4.86000000e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e-04, 3.50000000e-04, 0.00000000e+00],
       [7.50000000e-02, 3.00030000e-02, 1.47151620e-01, 4.87533000e-03,
        3.73810800e-03, 7.50000000e-04, 2.25000000e-04, 5.77500000e-03,
        5.59200200e-03, 1.95195070e-02, 0.00000000e+00],
       [2.25000000e-02, 9.16776006e-03, 3.49785660e-03, 5.40398859e-02,
        5.06894598e-02, 3.37515000e-04, 3.73504050e-03, 2.17504950e-03,
        4.89105120e-03, 1.41671282e-02, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.13336506e-02]])

In [61]:
beta

array([[6.55932264e-03, 4.47112347e-02, 2.02424452e-02, 1.93615440e-02,
        4.21142400e-02, 2.28876170e-02, 1.10750300e-02, 1.82501000e-02,
        1.81209072e-03, 3.60000000e-02, 0.00000000e+00],
       [4.55760160e-03, 3.19602200e-03, 5.05002600e-02, 6.80002000e-02,
        1.40002000e-01, 6.50000000e-04, 3.50000000e-03, 4.50000000e-02,
        1.14240040e-03, 1.20000000e-01, 0.00000000e+00],
       [8.00000000e-05, 1.10000000e-03, 1.30000000e-02, 1.00000000e-02,
        1.00000000e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        2.00000000e-05, 4.00000000e-01, 1.00000000e+00],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00]])

In [62]:
lik

0.011333650594090802

In [63]:
posterior_probs

array([[1.44686890e-01, 5.91749777e-01, 1.60744330e-02, 6.57704627e-02,
        1.80590715e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.59885882e-05, 1.11173358e-03, 0.00000000e+00],
       [3.01597545e-02, 8.46066740e-03, 6.55675328e-01, 2.92512472e-02,
        4.61759953e-02, 4.30135018e-05, 6.94833490e-05, 2.29295052e-02,
        5.63658220e-04, 2.06671347e-01, 0.00000000e+00],
       [1.58819083e-04, 8.89787097e-04, 4.01213497e-03, 4.76809175e-02,
        4.47247420e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        8.63102521e-06, 5.00002291e-01, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [64]:
alpha_scaled

array([[5.03473970e-01, 3.02084382e-01, 1.81250629e-02, 7.75349914e-02,
        9.78753398e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        2.01389588e-04, 7.04863559e-04, 0.00000000e+00],
       [2.56296726e-01, 1.02528942e-01, 5.02859713e-01, 1.66604149e-02,
        1.27741979e-02, 2.56296726e-03, 7.68890178e-04, 1.97348479e-02,
        1.91094907e-02, 6.67038099e-02, 0.00000000e+00],
       [1.36197932e-01, 5.54946648e-02, 2.11733704e-02, 3.27116475e-01,
        3.06835537e-01, 2.04305977e-03, 2.26091018e-02, 1.31660997e-02,
        2.96067136e-02, 8.57570467e-02, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [65]:
gamma

array([[2.5000000e-01, 1.5000000e-01, 9.0000000e-03, 3.8500000e-02,
        4.8600000e-02, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        1.0000000e-04, 3.5000000e-04, 0.0000000e+00],
       [7.5000000e-02, 3.0000000e-02, 1.3095000e-01, 3.3000000e-03,
        2.3100000e-03, 7.5000000e-04, 2.2500000e-04, 5.7750000e-03,
        4.8600000e-03, 1.7010000e-02, 0.0000000e+00],
       [2.2500000e-02, 9.0000000e-03, 2.9100000e-03, 4.5832500e-02,
        3.7713600e-02, 1.8750000e-04, 3.2737500e-03, 1.4437500e-03,
        2.1870972e-03, 5.9582250e-03, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 3.7713600e-03]])

In [66]:
optimal_path

array([ 1,  2,  4, 10])

# Model B
![](ModelB.png)

In [67]:
L,M1,M2,M3,I1,I2,C,R,T=tuple(range(9))

In [68]:
#folded probabilities
A = np.zeros((9,9))
B = np.zeros((9,3))

In [69]:
S_folded = np.zeros(9)
S_folded[M1] = 0.7
S_folded[M2] = 0.1+0.1*0.8
S_folded[M3] = 0.1+0.1*0.2*0.5
S_folded[C] = 0.1*0.2*0.5*0.2
S_folded[R] = 0.1*0.2*0.5*0.7
S_folded[T] = 0.1*0.2*0.5*0.1

In [70]:
init = 0.5 * S_folded
init[L] = 0.5

In [71]:
A[L,:] = 0.4 * S_folded
A[L,L] = 0.6
A[M1,M2] = 0.97
A[M1,M3] = 0.01*0.5
A[M1,I1] = 0.01
M1_E = 0.01*0.5 + 0.01
A[M1,C] = 0.2 * M1_E
A[M1,R] = 0.7 * M1_E
A[M1,T] = 0.1 * M1_E
A[M2,M3] = 0.5
A[M2,I2] = 0.05
M2_E = 0.4 + 0.05
A[M2,C] = 0.2 * M2_E
A[M2,R] = 0.7 * M2_E
A[M2,T] = 0.1 * M2_E
A[M3,C] = 0.2 
A[M3,R] = 0.7 
A[M3,T] = 0.1 
A[I1,I1] = 0.5
A[I1,M2] = 0.5
A[I2,I2] = 0.1
A[I2,M3] = 0.9
A[C,:] = 0.1 * S_folded
A[C,C] = 0.9 + 0.1*0.1*0.2*0.5*0.2
A[R,R] = 0.6
A[R,T] = 0.4
A[T,T] = 1

In [72]:
B[:,:2] = 0.5
B[M2,:2] = (0.1, 0.9)
B[M3,:2] = (0.7, 0.3)
B[T,:] = (0,0,1)

In [73]:
np.testing.assert_almost_equal(np.sum(init), 1)
np.testing.assert_almost_equal(np.sum(A, -1), 1)
np.testing.assert_almost_equal(np.sum(B, -1), 1)

In [74]:
A,B,init

(array([[6.000e-01, 2.800e-01, 7.200e-02, 4.400e-02, 0.000e+00, 0.000e+00,
         8.000e-04, 2.800e-03, 4.000e-04],
        [0.000e+00, 0.000e+00, 9.700e-01, 5.000e-03, 1.000e-02, 0.000e+00,
         3.000e-03, 1.050e-02, 1.500e-03],
        [0.000e+00, 0.000e+00, 0.000e+00, 5.000e-01, 0.000e+00, 5.000e-02,
         9.000e-02, 3.150e-01, 4.500e-02],
        [0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
         2.000e-01, 7.000e-01, 1.000e-01],
        [0.000e+00, 0.000e+00, 5.000e-01, 0.000e+00, 5.000e-01, 0.000e+00,
         0.000e+00, 0.000e+00, 0.000e+00],
        [0.000e+00, 0.000e+00, 0.000e+00, 9.000e-01, 0.000e+00, 1.000e-01,
         0.000e+00, 0.000e+00, 0.000e+00],
        [0.000e+00, 7.000e-02, 1.800e-02, 1.100e-02, 0.000e+00, 0.000e+00,
         9.002e-01, 7.000e-04, 1.000e-04],
        [0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
         0.000e+00, 6.000e-01, 4.000e-01],
        [0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.0

In [75]:
alpha = forward(seq, init, A, B)
beta = backward(seq, init, A, B)
lik = likelihood_forward(alpha)
lik2 = likelihood_backward(beta, seq, init, B)
np.testing.assert_almost_equal(lik, lik2)
posterior_probs = posterior(alpha, beta, lik)
alpha_scaled, lik3 = forward(seq, init, A, B, scaled=True)
np.testing.assert_almost_equal(lik, lik3)
gamma = viterbi_variables(seq, init, A, B)
optimal_path = viterbi_path(gamma, A)

In [76]:
alpha

array([[2.50000000e-01, 1.75000000e-01, 9.00000000e-03, 3.85000000e-02,
        0.00000000e+00, 0.00000000e+00, 5.00000000e-04, 1.75000000e-03,
        0.00000000e+00],
       [7.50000000e-02, 3.50175000e-02, 1.68983100e-01, 4.91415000e-03,
        8.75000000e-04, 2.25000000e-04, 4.84255000e-03, 1.66864250e-02,
        0.00000000e+00],
       [2.25000000e-02, 1.06694892e-02, 3.98916409e-03, 6.17556839e-02,
        3.93837500e-04, 4.23582750e-03, 1.03578125e-02, 3.36312550e-02,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.98336228e-02]])

In [77]:
beta

array([[1.06095920e-02, 8.64108263e-02, 4.15254829e-02, 4.21088510e-02,
        4.46645250e-02, 4.09527000e-02, 2.82017450e-03, 3.60000000e-02,
        0.00000000e+00],
       [4.29404000e-03, 6.81515000e-03, 9.80045000e-02, 1.40010000e-01,
        2.25000000e-03, 6.30000000e-02, 1.08851000e-03, 1.20000000e-01,
        0.00000000e+00],
       [4.00000000e-04, 1.50000000e-03, 4.50000000e-02, 1.00000000e-01,
        0.00000000e+00, 0.00000000e+00, 1.00000000e-04, 4.00000000e-01,
        1.00000000e+00],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00]])

In [78]:
lik

0.0198336227946755

In [79]:
posterior_probs

array([[1.33732401e-01, 7.62437340e-01, 1.88432215e-02, 8.17395178e-02,
        0.00000000e+00, 0.00000000e+00, 7.10957985e-05, 3.17642423e-03,
        0.00000000e+00],
       [1.62377294e-02, 1.20325731e-02, 8.35001472e-01, 3.46900891e-02,
        9.92632572e-05, 7.14695452e-04, 2.65769101e-04, 1.00958409e-01,
        0.00000000e+00],
       [4.53774890e-04, 8.06924385e-04, 9.05091248e-03, 3.11368652e-01,
        0.00000000e+00, 0.00000000e+00, 5.22235025e-05, 6.78267513e-01,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00]])

In [80]:
alpha_scaled

array([[5.26592944e-01, 3.68615061e-01, 1.89573460e-02, 8.10953133e-02,
        0.00000000e+00, 0.00000000e+00, 1.05318589e-03, 3.68615061e-03,
        0.00000000e+00],
       [2.44663302e-01, 1.14233296e-01, 5.51252843e-01, 1.60308289e-02,
        2.85440519e-03, 7.33989906e-04, 1.57972570e-02, 5.44340779e-02,
        0.00000000e+00],
       [1.52508180e-01, 7.23193062e-02, 2.70391181e-02, 4.18588754e-01,
        2.66948624e-03, 2.87110375e-02, 7.02067172e-02, 2.27957400e-01,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00]])

In [81]:
gamma

array([[2.50000000e-01, 1.75000000e-01, 9.00000000e-03, 3.85000000e-02,
        0.00000000e+00, 0.00000000e+00, 5.00000000e-04, 1.75000000e-03,
        0.00000000e+00],
       [7.50000000e-02, 3.50000000e-02, 1.52775000e-01, 3.30000000e-03,
        8.75000000e-04, 2.25000000e-04, 3.85000000e-03, 1.34750000e-02,
        0.00000000e+00],
       [2.25000000e-02, 1.05000000e-02, 3.39500000e-03, 5.34712500e-02,
        2.18750000e-04, 3.81937500e-03, 6.87487500e-03, 2.40620625e-02,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        9.62482500e-03]])

In [82]:
optimal_path

array([1, 2, 7, 8])